In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = 'defog/sqlcoder-7b'

peft_model_id = "/home/ksaff/Desktop/ttyd/fine_tuning/1st_try"
config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
                                load_in_8bit=True,
                                bnb_8bit_use_double_quant=True,
                                bnb_8bit_quant_type="nf4",
                                bnb_8bit_compute_dtype=torch.bfloat16
                                )

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, return_dict=True, quantization_config=bnb_config, device_map={'': 0}
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

/home/ksaff/miniconda3/envs/llm_jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ksaff/.cache/huggingface/hub/models--defog--sqlcoder-7b/snapshots/cb59ea29ef43769d221b98a8ebe81a0175c76658/pytorch_model.bin.index.json
['/home/ksaff/.cache/huggingface/hub/models--defog--sqlcoder-7b/snapshots/cb59ea29ef43769d221b98a8ebe81a0175c76658/pytorch_model-00001-of-00002.bin', '/home/ksaff/.cache/huggingface/hub/models--defog--sqlcoder-7b/snapshots/cb59ea29ef43769d221b98a8ebe81a0175c76658/pytorch_model-00002-of-00002.bin']


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.14s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
import os

prompt = 'Make SQLite query based on instruction, for SQL table Salaries, with columns: id, EmployeeName, JobTitle, BasePay, OvertimePay, OtherPay, Benefits, TotalPay, TotalPayBenefits, Year. '
instruction = "Query 5 most paid jobs"
text = (
    prompt
    + '### Instruction: '
    + instruction
    + '### Usage: '
)

In [46]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.to('cuda')

In [47]:
inputs = {
    'input_ids': input_ids,
    'max_length': 200,
    'max_time': 5,
}

output = model.generate(**inputs)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Make SQLite query based on instruction, for SQL table Salaries, with columns: id, EmployeeName, JobTitle, BasePay, OvertimePay, OtherPay, Benefits, TotalPay, TotalPayBenefits, Year. ### Instruction: Query 5 most paid jobs### Usage: 5### Output: JobTitle, TotalPay, TotalPayBenefits, Year### Sample Output: JobTitle, TotalPay, TotalPayBenefits, Year### JobTitle, TotalPay, TotalPayBenefits, Year### JobTitle, Total'

In [48]:
import re

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

match = re.search(r'### Response:\s*(.*)', generated_text)
# response = match.group(1)
# response
response = None

In [49]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()

conn = sqlite3.connect('/home/ksaff/Desktop/sql/choosen_DB.sqlite')
c = conn.cursor()

def execute_query(query):

    c.execute(query)
    result = c.fetchall()

    # Extract column names
    columns = [desc[0] for desc in c.description]
    
    # Prepare the response
    response = {"query": query, "headers": columns, "data": result}

    return response

In [50]:
result = execute_query('SELECT AVG(TotalPay) AS AverageTotalPay, EmployeeName, JobTitle FROM Salaries GROUP BY EmployeeName, JobTitle ORDER BY AverageTotalPay DESC LIMIT 5;')
print(f'User instruction: {instruction}')
print(f'model output: {response}')
'Result query:', result['data']

User instruction: Query 5 most paid jobs
model output: None


('Result query:',
 [(567595.43,
   'NATHANIEL FORD',
   'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY'),
  (538909.28, 'GARY JIMENEZ', 'CAPTAIN III (POLICE DEPARTMENT)'),
  (390111.98, 'Amy P Hart', 'Asst Med Examiner'),
  (373849.97, 'David Shinn', 'Deputy Chief 3'),
  (362844.66, 'Gary Altenberg', 'Lieutenant, Fire Suppression')])